We start by importing modules we are going to use

In [343]:
import googlemaps
import geocoder
import pandas as pd
from datetime import datetime
import numpy as np
import urllib.parse

Some Constants and Parameters that are defined here

In [344]:
CUISINE_LIST = [
    "Italian", "Pizza", "Pasta",
    "Mexican", "Taco", "Burrito",
    "Chinese", "Dimsum", "Noodles",
    "Japanese", "Sushi", "Ramen",
    "Indian", "Curry", "Biryani",
    "Thai", "Pad Thai", "Tom Yum",
    "Greek", "Gyro", "Moussaka",
    "French", "Baguette", "Croissant",
    "Spanish", "Paella", "Tapas",
    "American", "Burger", "Steak",
    "Middle Eastern", "Falafel", "Shawarma",
    "Vietnamese", "Pho", "Banh Mi"
]
SEARCH_RADIUS=500
SEARCH_TYPE='restaurant'
KEYWORDS=['food truck', 'food cart']

Then we get a google maps client using our API_KEY

In [345]:
# API_KEY 
API_KEY = ''
gmaps = googlemaps.Client(key=API_KEY)

Starting by getting the current location, then getting today's weekday and then starting the search process using the defined parameters to get all the nearby places of interest

In [346]:
location=geocoder.ip('me').latlng
location_s=gmaps.reverse_geocode(location)[0]['formatted_address']
today= datetime.now().strftime('%A')
results = gmaps.places_nearby(location=location, radius=SEARCH_RADIUS, type=SEARCH_TYPE, keyword=KEYWORDS)

Here we are getting place_id which could be used then to get more details about the places we found

In [347]:
results = [result['place_id'] for result in results['results']]

Creating a dataframe

In [348]:
df = pd.DataFrame({'place_id':results})

Using pandas.apply and lambda to get the fields we need from the gmaps response we got earlier

In [349]:
df['place_details'] = df['place_id'].apply(lambda x: gmaps.place(place_id=x, 
                                                                 fields=['name','formatted_address','rating','website', 'opening_hours'])['result'] if gmaps.place(place_id=x) else None)

Here we are dropping irrelevant columns and renaming existing ones to be more convienient

In [350]:
df=df.join(pd.json_normalize(df['place_details']),how='right').drop(['place_id', 'place_details', 'opening_hours.open_now', 'opening_hours.periods'], axis=1)
df = df.rename(columns={'name':'Name','formatted_address':'Address','rating':'Rating','website':'Website', 'opening_hours.weekday_text':'Hours'})
df = df[['Name', 'Address', 'Rating', 'Website', 'Hours']]

Trying to deduce cuisine from the name of the restaurant as there were no data that could be used from google maps that could tell which cuisine is which

In [351]:
df['Cuisine'] = df['Name'].apply(lambda name: next((cuisine for cuisine in CUISINE_LIST if cuisine.lower() in name.lower()), None))
df.head(3)

,Name,Address,Rating,Website,Hours,Cuisine
0,Sweet's Turkey BBQ and Catering (Food Truck),"Greensboro, NC 27401, USA",4.1,NaN,NaN,None
1,Maho's Bistro,"200 S Greene St, Greensboro, NC 27406, USA",4.3,NaN,NaN,None
2,Skeel's Ice Cream Trucks,"Greensboro, NC 27401, USA",5.0,http://www.skeelsicecreamtrucks.com/,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",None


Method for converting google maps format of opening hours, to make it more readable and convienient to reader

In [352]:
def convert_to_today_open_hours(x):
    today = datetime.now().strftime('%A')
    if x is np.nan:
        return x
    for i in x:
        if today in i:
            return i[len(today)+1:]
    return np.nan

In [353]:
df['Hours'] = df['Hours'].apply(convert_to_today_open_hours)
df.head(10)

,Name,Address,Rating,Website,Hours,Cuisine
0,Sweet's Turkey BBQ and Catering (Food Truck),"Greensboro, NC 27401, USA",4.1,NaN,NaN,None
1,Maho's Bistro,"200 S Greene St, Greensboro, NC 27406, USA",4.3,NaN,NaN,None
2,Skeel's Ice Cream Trucks,"Greensboro, NC 27401, USA",5.0,http://www.skeelsicecreamtrucks.com/,Open 24 hours,None
3,"International Food, LLC","114 N Elm St #108, Greensboro, NC 27401, USA",4.8,NaN,Closed,None
4,Crafted The Art of the Taco,"220 S Elm St, Greensboro, NC 27401, USA",4.4,http://www.eatatcrafted.com/,11:30 AM – 9:00 PM,Taco


Exporting the dataframe into a csv file

In [354]:
df.to_csv('food_trucks_around_me.csv', mode='w')

A method to get all the relevant details of a place of interest, like directions, traveling time, distance, and if the distance is less than a mile, the transportation method is set to be walking otherwise it's driving.

In [355]:
def get_travel_details(x):
    location=geocoder.ip('me').latlng
    location_s=gmaps.reverse_geocode(location)[0]['formatted_address']
    directions = gmaps.directions(location_s, x)
    distance = directions[0]['legs'][0]['distance']['text']
    transportation= 'walking' if float(distance.split(' ')[0]) < 1.0 else 'driving'
    directions= gmaps.directions(location, x, mode=transportation)
    travel_time=directions[0]['legs'][0]['duration']['text']
    return pd.Series([distance, transportation, travel_time])

Grouping everything together to have something convienient to the user into a single dataframe

In [356]:
plan = df[['Name', 'Address', 'Cuisine', 'Hours']]
plan[['Travel Distance', 'Transportation Type', 'Travel Time']] = plan['Address'].apply(get_travel_details)
plan.head()

C:\Users\mosta\AppData\Local\Temp\ipykernel_15996\416366572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plan[['Travel Distance', 'Transportation Type', 'Travel Time']] = plan['Address'].apply(get_travel_details)


,Name,Address,Cuisine,Hours,Travel Distance,Transportation Type,Travel Time
0,Sweet's Turkey BBQ and Catering (Food Truck),"Greensboro, NC 27401, USA",None,NaN,2.6 mi,driving,6 mins
1,Maho's Bistro,"200 S Greene St, Greensboro, NC 27406, USA",None,NaN,0.1 mi,walking,2 mins
2,Skeel's Ice Cream Trucks,"Greensboro, NC 27401, USA",None,Open 24 hours,2.6 mi,driving,6 mins
3,"International Food, LLC","114 N Elm St #108, Greensboro, NC 27401, USA",None,Closed,0.1 mi,walking,3 mins
4,Crafted The Art of the Taco,"220 S Elm St, Greensboro, NC 27401, USA",Taco,11:30 AM – 9:00 PM,0.2 mi,walking,5 mins


Exporting the dataframe to a csv file

In [357]:
plan.to_csv('plan.csv', mode='w')

Using the furthest place of interest as a destination, and using the rest of the places as waypoints, we can figure the optimized directions to visit all the suggested places around us.

In [358]:
# directions=gmaps.directions(location,))
idx=plan['Travel Distance'].idxmax()
destination=plan.loc[idx, 'Name']

This function is made to generate a url to get a google maps link that could be used for navigating

In [359]:
def generate_gmaps_url(origin, destination, waypoints=None, mode='driving'):
    prefix="https://www.google.com/maps/dir/?api=1"
    origin=urllib.parse.quote_plus(origin)
    destination = urllib.parse.quote_plus(destination)
    url = f"{prefix}&origin={origin}&destination={destination}"
    if waypoints is not None:
        waypoints = '|'.join(urllib.parse.quote_plus(waypoint) for waypoint in waypoints)
        url += f"&waypoints={waypoints}"
    url += f"&travelmode={mode}"
    return url

Final output, a list of directions, and a link for the direction via google maps viewport.

In [360]:
directions_result = gmaps.directions(location_s, destination, waypoints=plan['Name'].to_list(), optimize_waypoints=True)
for i, leg in enumerate(directions_result[0]['legs']):
    print(f"Leg {i+1}:")
    print(f"  Start: {leg['start_address']}")
    print(f"  End: {leg['end_address']}")
    print(f"  Distance: {leg['distance']['text']}")
    print(f"  Duration: {leg['duration']['text']}")

print(generate_gmaps_url(location_s, destination, plan['Name'].to_list(), mode='driving' if plan['Transportation Type'].any() == 'driving' else 'walking'))

Leg 1:
  Start: 216 W Market St, Greensboro, NC 27401, USA
  End: Greensboro, NC 27401, USA
  Distance: 20 ft
  Duration: 1 min
Leg 2:
  Start: Greensboro, NC 27401, USA
  End: Greensboro, NC 27401, USA
  Distance: 3 ft
  Duration: 1 min
Leg 3:
  Start: Greensboro, NC 27401, USA
  End: 200 S Greene St, Greensboro, NC 27406, USA
  Distance: 0.1 mi
  Duration: 1 min
Leg 4:
  Start: 200 S Greene St, Greensboro, NC 27406, USA
  End: 220 S Elm St, Greensboro, NC 27401, USA
  Distance: 0.2 mi
  Duration: 1 min
Leg 5:
  Start: 220 S Elm St, Greensboro, NC 27401, USA
  End: 114 N Elm St #108, Greensboro, NC 27401, USA
  Distance: 0.2 mi
  Duration: 1 min
Leg 6:
  Start: 114 N Elm St #108, Greensboro, NC 27401, USA
  End: Greensboro, NC 27401, USA
  Distance: 0.4 mi
  Duration: 2 mins
https://www.google.com/maps/dir/?api=1&origin=216+W+Market+St%2C+Greensboro%2C+NC+27401%2C+USA&destination=Sweet%27s+Turkey+BBQ+and+Catering+%28Food+Truck%29&waypoints=Sweet%27s+Turkey+BBQ+and+Catering+%28Food+Tru